In [2]:
import pickle

import pandas as pd

import numpy as np

from sklearn import datasets

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier

from skl2onnx import convert_sklearn

from skl2onnx.common.data_types import FloatTensorType

In [3]:
iris = datasets.load_iris()
data = pd.DataFrame(
    {
        "sepalLength": iris.data[:, 0],
        "sepalWidth": iris.data[:, 1],
        "petalLength": iris.data[:, 2],
        "petalWidth": iris.data[:, 3],
        "species": iris.target,
    }
)
data.head()

,sepalLength,sepalWidth,petalLength,petalWidth,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [4]:
X = data[["sepalLength", "sepalWidth", "petalLength", "petalWidth"]]  # Features
y = data["species"]  # Labels

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3
)  # 70% training and 30% test

type(X_train)

pandas.core.frame.DataFrame

In [5]:
# Create a Gaussian Classifier
model = RandomForestClassifier(n_estimators=100)

# Train the model using the training sets y_pred=clf.predict(X_test)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [6]:
print(X_train)

     sepalLength  sepalWidth  petalLength  petalWidth
56           6.3         3.3          4.7         1.6
140          6.7         3.1          5.6         2.4
126          6.2         2.8          4.8         1.8
101          5.8         2.7          5.1         1.9
71           6.1         2.8          4.0         1.3
..           ...         ...          ...         ...
93           5.0         2.3          3.3         1.0
110          6.5         3.2          5.1         2.0
68           6.2         2.2          4.5         1.5
62           6.0         2.2          4.0         1.0
97           6.2         2.9          4.3         1.3

[105 rows x 4 columns]


In [7]:
model.predict([[5, 3, 1.6, 0.2]])

/home/troyer/code/mlops-demo/mlops-demo-iris-training-service/.venv/lib64/python3.9/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([0])

In [11]:
# save the model to disk
model_location = "../models"

filename = f"{model_location}/iris-model.pkl"
pickle.dump(model, open(filename, "wb"))

# some time later...

# load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)
# print(result)

In [12]:
initial_type = [('float_input', FloatTensorType([None, 4]))]
onx = convert_sklearn(model, initial_types=initial_type)

In [ ]:
onnx_filename = f"{model_location}/iris-model.onnx"

with open(onnx_filename, "wb") as f:
    f.write(onx.SerializeToString())